<a href="https://www.kaggle.com/code/valerybonneau/ps4e01-eda-fe-nn-ensemble-ml?scriptVersionId=193455759" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


# Table of content
0. [Introduction](#zero)
1. [Libraries and data import](#one)
2. [EDA](#two)
3. [Data Preparation](#three)
4. [Feature Engineering](#four)
5. [Models Training](#five)
6. [NN Focus](#six)
7. [ensemble](#seven)
8. [Conclusion and Submission](#eight)

<a id='zero'></a>
# 0. Introduction
It is a very quick and dirty implementation. I'm running AutoMl without the slightest EDA. It will give me a baseline that I'll use for my other notebooks.<br>
The first test gives 0.89094 and a temporary eleventh place on the lb.

## A. Problem Scope
The goal is to predict if a customer will leave or not his or her bank.The data are taken from that [Bank Customer Churn Dataset](https://www.kaggle.com/datasets/shubhammeshram579/bank-customer-churn-prediction). 

## B. Personal Goal
I have created a [notebook](https://www.kaggle.com/code/valerybonneau/ps4e01-automl-test-0-89094-lb-score) that includes an AutoML submission and gave a public score of 0.89094. My plan is to focus on data preparation, feature engineering and neural network to improve that score and ideally, end up in the top 15%.

## C. Acknowledgments
I have been through a lot of notebooks and read or contribute to discussions. Some notebooks or people really did help me a lot (on top of the community). Thanks to... Have a look a their notebooks!

Good luck with the competition!

<a id='one'></a>
# 1. Libraries and data import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e1/sample_submission.csv
/kaggle/input/playground-series-s4e1/train.csv
/kaggle/input/playground-series-s4e1/test.csv


In [2]:
pip install scikit-learn==1.3.2

  Obtaining dependency information for scikit-learn==1.3.2 from https://files.pythonhosted.org/packages/d0/0b/26ad95cf0b747be967b15fb71a06f5ac67aba0fd2f9cd174de6edefc4674/scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 47.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

import time

# math and stats
import scipy.stats as stats
from scipy.stats import randint, uniform
import tensorflow_probability as tfp
from scipy.stats import boxcox

# Preprocessing
import sklearn
from sklearn.model_selection import cross_validate, KFold, RandomizedSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import mutual_info_regression

# Simple Model
from sklearn.linear_model import LogisticRegression, Ridge, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeClassifier

# ensemble models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Neural network model
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout

# Models 
import xgboost as xgb
from xgboost import XGBClassifier

# Metrics
from sklearn.metrics import roc_auc_score


# lOOK AND FEEL
sns.set(style='darkgrid')
sns.set_palette(sns.color_palette("Set2"))

pd.options.display.max_rows = None
pd.options.display.max_columns = None

# Global Variable
random_seed = 73
epsilon = 1e-7

## Dataset Description
The following description is taken from this [notebook](https://www.kaggle.com/datasets/shubhammeshram579/bank-customer-churn-prediction). <br>The bank customer churn dataset is a commonly used dataset for predicting customer churn in the banking industry. It contains information on bank customers who either left the bank or continue to be a customer. The dataset includes the following attributes:

- **Customer ID**: A unique identifier for each customer
- **Surname**: The customer's surname or last name
- **Credit Score**: A numerical value representing the customer's credit score
- **Geography**: The country where the customer resides (France, Spain or Germany)
- **Gender**: The customer's gender (Male or Female)
- **Age**: The customer's age.
- **Tenure**: The number of years the customer has been with the bank
- **Balance**: The customer's account balance
- **NumOfProducts**: The number of bank products the customer uses (e.g., savings account, credit card)
- **HasCrCard**: Whether the customer has a credit card (1 = yes, 0 = no)
- **IsActiveMember**: Whether the customer is an active member (1 = yes, 0 = no)
- **EstimatedSalary**: The estimated salary of the customer
- **Exited**: Whether the customer has churned (1 = yes, 0 = no)

In [4]:
train = pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')
train.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [5]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,165034.0,8.251650e+04,47641.356500,0.00,41258.25,82516.5,1.237748e+05,165033.00
CustomerId,165034.0,1.569201e+07,71397.816791,15565701.00,15633141.00,15690169.0,1.575682e+07,15815690.00
CreditScore,165034.0,6.564544e+02,80.103340,350.00,597.00,659.0,7.100000e+02,850.00
Age,165034.0,3.812589e+01,8.867205,18.00,32.00,37.0,4.200000e+01,92.00
Tenure,165034.0,5.020353e+00,2.806159,0.00,3.00,5.0,7.000000e+00,10.00
Balance,165034.0,5.547809e+04,62817.663278,0.00,0.00,0.0,1.199395e+05,250898.09
NumOfProducts,165034.0,1.554455e+00,0.547154,1.00,1.00,2.0,2.000000e+00,4.00
HasCrCard,165034.0,7.539537e-01,0.430707,0.00,1.00,1.0,1.000000e+00,1.00
IsActiveMember,165034.0,4.977702e-01,0.499997,0.00,0.00,0.0,1.000000e+00,1.00
EstimatedSalary,165034.0,1.125748e+05,50292.865585,11.58,74637.57,117948.0,1.551525e+05,199992.48


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [7]:
print(f'The number of NA for the train set is {train.isna().sum().sum()}')
print(f'The number of NA for the test set is {test.isna().sum().sum()}')

The number of NA for the train set is 0
The number of NA for the test set is 0


<a id='two'></a>
# 2. EDA

In [ ]:
customers = train.copy().drop(columns=['id', 'Surname'])
features = customers.columns.tolist()
target = train[features.pop()]

In [ ]:
feat_num = ['CustomerId', 'CreditScore', 'Age', 'Balance', 'EstimatedSalary']
feat_cat = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'Tenure', 'NumOfProducts']

## Distribution
Not too sure about what needs to be done here. `CreditScore` has a normal distribution, `Age` is a little bit right skewed but if I remove some outliers, that should  be enough. `Balance` is more problematic with its high number of zeros. Also, I guess I should remove `CustomerId` but it could contain indirect information about the tenure.

In [ ]:
_, axes = plt.subplots(nrows=2, ncols=3, figsize=(18,12))
plt.suptitle('Distribution of numerical features')
for feat,ax in zip(feat_num, axes.ravel()):
    sns.violinplot(data=customers, x='Exited', y=feat, ax=ax)
axes[-1,-1].set_visible(False)

In [ ]:
_, axes = plt.subplots(nrows=3, ncols=2, figsize=(15,12))
for feat, ax in zip(feat_cat, axes.ravel()):
    sns.countplot(data=customers, x=feat, ax=ax, hue='Exited')
    ax.set_title(f'{feat} vs Exited')
plt.tight_layout()
plt.show()

## Outliers

In [ ]:
_, axes = plt.subplots(nrows=2, ncols=3, figsize=(12,10))
plt.suptitle('Outliers detection')
for feat, ax in zip(feat_num, axes.ravel()):
    sns.boxplot(data=customers, x='Exited', y=feat, ax=ax)
    ax.set_xlabel('')
axes[-1,-1].set_visible(False)
plt.tight_layout()


## Correlation Matrix
I've rarely seen dataset with such a low correlation between features and target. At first glance, it's like no feature has any impact on the target. The feature engineering might be challenging.

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})

col_not_object = customers.select_dtypes(exclude='object')
customers_corr = customers[col_not_object.columns]
corr = customers_corr.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(100, 7, s = 75, l = 40, n = 20, center = 'light', as_cmap = True)

sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, square=True, linewidths=.5, cbar_kws={'shrink':.5}, annot=True, fmt='.1f')

In [ ]:
g = sns.pairplot(customers.sample(frac=.02),
                 hue='Exited',
                 corner=True
                )

## Class Distribution

In [ ]:
plt.figure(figsize=(5,5))
sns.countplot(data=customers, x='Exited')
class_weight = (customers['Exited'].value_counts(normalize=True)*100).round(2).tolist()
for idx, value in enumerate(class_weight):
    plt.text(idx, 10000, f'{value:.2f}%', color='white', ha='center', fontweight='bold')

In [ ]:
%%time

from ydata_profiling import ProfileReport
    
# Generate the profile report
profile = ProfileReport(train, title='PS4E01 - Automated Report')
profile.to_notebook_iframe()

In [ ]:
# plt.step(time, survival_prob, where='post')
# plt.xlabel('Tenure (years)')
# plt.ylabel('Probability to Exit (%)')
# plt.suptitle('Probability to Exit with Tenure evolution')

<a id='three'></a>
# 3. Data Preparation

In [8]:
def normalize_features(X1, X2, feat_tr):
    """
    Normalize features from X1 and X2 using the appropriate transformation 
            
    Args:
        X1 (dataframe): dataframe corresponding to X_train
        X2 (dataframe): dataframe corresponding to X_test
        feat_tr: list of features to treat.
        
    Returns:
        X1, X2: the dataframes after normalization of their features.
    """
    X1 = X1.copy()
    X2 = X2.copy()
    
    # I treat only right skewed for now
    for feat in feat_tr:
        X1[feat] = boxcox(X1[feat]+epsilon, lmbda=0.05)
        X2[feat] = boxcox(X2[feat]+epsilon, lmbda=0.05)
            
    return X1, X2

In [24]:
def scale_features(X1, X2=pd.DataFrame()):
    """
    Scale X1 and X2. 
    Apply fit_transform to X1 and transform to X2 and returns the result
    
    Args:
        X1 (dataframe): dataframe corresponding to X_train
        X2 (dataframe): dataframe corresponding to X_test
        
    Returns:
        X1, X2: the dataframes after scaling.
    """
    
    X1 = X1.copy()
    X2 = X2.copy()
    
    cols = X1.columns
    scaler= MinMaxScaler()
    
    X1 = scaler.fit_transform(X1)
    X2 = scaler.transform(X2)

    X1 = pd.DataFrame(data=X1, columns=cols)
    X2 = pd.DataFrame(data=X2, columns=cols)
    
    return X1, X2

## X, y creation

In [33]:
X_train = train.drop(columns=['id', 'CustomerId', 'Exited'])
X_test = test.drop(columns=['id', 'CustomerId'])
y_train = train['Exited']

## Normalization

## Encoding
Initially, I thought there was only `Geography` and `Gender` to encode. I use one-hot encoding for `Geography`.
Now, I will make a last try with `Surname` and use binary encoding.

In [34]:
X_train = pd.get_dummies(X_train, columns=['Geography'], prefix=['Country'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['Geography'], prefix=['Country'], drop_first=True)

X_train['Gender'] = X_train['Gender'].map({'Male':0, 'Female':1}).astype('bool')
X_test['Gender'] = X_test['Gender'].map({'Male':0, 'Female':1}).astype('bool')

from category_encoders import BinaryEncoder
be = BinaryEncoder(cols=['Surname'])
train_surname = be.fit_transform(X_train['Surname'])
test_surname = be.transform(X_test['Surname'])

X_train = pd.concat([X_train, train_surname], axis=1 ) 
X_test = pd.concat([X_test, test_surname], axis=1)

X_train = X_train.drop(columns=['Surname'])
X_test = X_test.drop(columns=['Surname'])

In [35]:
feat_tr = ['Age', 'CreditScore', 'Balance', 'EstimatedSalary']

<a id='four'><a/>
# 4. Feature Engineering
Under construction

In [37]:
def add_feature(X1, feat_list):
    """
    Add new features to X1 for each feature contained in the feat_list
    
    Args:
        X1 (dataframe): the dataframe to remove outliers from
        feat_list: a list that contains all the feature to work on
         
    Returns:
        X1: the dataframes with the new features
    """
    epsilon = 1e-7
    X = X1.copy()
    for feat in feat_list:
        X[str(feat) + '_square'] = np.square(X[feat])
#         X[str(feat) + '_cube'] = np.power(X[feat],3)
        X[str(feat) + '_log'] = np.log1p(X[feat] + epsilon)
        
    return X

In [38]:
from itertools import combinations, product

def add_combination_feature(X, feat_list):
    """
    Add new features to X1 for each feature contained in the feat_list based on the other feature of X1
    
    Args:
        X1 (dataframe): the dataframe to remove outliers from
        feat_list: a list that contains all the feature to work on
         
    Returns:
        X1: the dataframes with the new features
    """
    epsilon = 1e-7
    new_columns = []
    
    X1 = X.copy()

    for feat1, feat2 in product(feat_list, repeat=2):
        if feat1 != feat2:
            new_col_name_div = f'{feat1}_div_{feat2}'
            X1[new_col_name_div] = X1[feat1] / (X1[feat2]+epsilon)
            
            new_col_name_times = f'{feat1}_times_{feat2}'
            X1[new_col_name_times] = X1[feat1] * X1[feat2]
            
    return X1

In [39]:
# Adapted from the 'feature engineering' kaggle course: https://www.kaggle.com/code/ryanholbrook/feature-engineering-for-house-prices/notebook
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, random_state=random_seed)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [40]:
features = X_train.columns
X_train_fi, X_test_fi = scale_features(X_train, X_test)

mi_scores0 = make_mi_scores(X_train_fi, y_train, features)
mi_scores0

NumOfProducts      0.089390
Age                0.082859
IsActiveMember     0.023768
Country_Germany    0.017628
Balance            0.015483
Gender             0.012631
EstimatedSalary    0.008355
CreditScore        0.004411
Surname_5          0.003197
Surname_1          0.003143
Surname_3          0.002395
Country_Spain      0.002226
Tenure             0.001383
Surname_2          0.000951
Surname_7          0.000810
Surname_6          0.000742
Surname_10         0.000631
Surname_0          0.000576
HasCrCard          0.000343
Surname_9          0.000049
Surname_4          0.000000
Surname_8          0.000000
Surname_11         0.000000
Name: MI Scores, dtype: float64

In [ ]:
mi_scores0.to_csv('mi_scores0.csv', index=True, sep=',')

In [ ]:
feat_l = ['Age', 'CreditScore', 'Tenure', 'Balance', 'EstimatedSalary', 'NumOfProducts']

X_train_f = add_combination_feature(X_train, feat_l)
X_test_f = add_combination_feature(X_test, feat_l)

X_train_c = add_feature(X_train_f, feat_l)
X_test_c = add_feature(X_test_f, feat_l)

In [ ]:
features = X_train_c.columns
X_train_fi, X_test_fi = scale_features(X_train_c, X_test_c)

mi_scores1 = make_mi_scores(X_train_fi, y_train, features)
mi_scores1.to_csv('mi_scores1.csv', index=True, sep=',')
mi_scores1

In [ ]:
X_train_f = add_combination_feature(X_train_c, feat_l)
X_test_f = add_combination_feature(X_test_c, feat_l)

X_train_f.head()

In [ ]:
features = X_train.columns
X_train_fi, X_test_fi = scale_features(X_train_f, X_test_f)

mi_scores = make_mi_scores(X_train_fi, y_train, features)
mi_scores.to_csv('mi_scores.csv', index=True, sep=',')

In [ ]:
features_best_scores = mi_scores[:50].index

## SHAP analysis
I'm not very fgood at interpreting feature importance or feature contribution so I'll spend a bit of time on SHAP.

In [ ]:
import shap

# I'm using the parameters and the model that best performed so far with a score of 0.8902554795195133
params_shap = {'n_estimators': 1464,
                      'max_depth': 41,
                      'learning_rate': 0.03433,
                      'min_child_weight': 2.96035,
                      'min_child_samples': 30,
                      'subsample': 0.878299,
                      'subsample_freq': 3,
                      'colsample_bytree': 0.501275,
                      'num_leaves': 25}
model_shap = LGBMClassifier(random_state=random_seed,verbose=-1, **params_shap).fit(X_train, y_train)
explainer = shap.TreeExplainer(model_shap)

In [ ]:
X_sampled = X_train.sample(1000, random_state=73)
shap_values = explainer.shap_values(X_sampled)
shap.summary_plot(shap_values[1], X_sampled)

In [ ]:
shap.initjs()

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1], X_sampled)

In [ ]:
model_shap2 = LGBMClassifier(random_state=random_seed,verbose=-1, **params_shap).fit(X_train_c, y_train)
explainer = shap.TreeExplainer(model_shap2)

In [ ]:
X_sampled = X_train_c.sample(1000, random_state=73)
shap_values = explainer.shap_values(X_sampled)
shap.summary_plot(shap_values[1], X_sampled)

In [ ]:
model_shap3 = LGBMClassifier(random_state=random_seed,verbose=-1, **params_shap).fit(X_train_f, y_train)
explainer = shap.TreeExplainer(model_shap3)

In [ ]:
X_sampled = X_train_f.sample(1000, random_state=73)
shap_values = explainer.shap_values(X_sampled)
shap.summary_plot(shap_values[1], X_sampled)

In [ ]:
X_train_c.shape

<a id='five'><a/>
# 5. Model Training
Under construction

In [ ]:
from sklearn.metrics import log_loss
# adapted from https://www.kaggle.com/code/ambrosm/pss3e23-eda-which-makes-sense
# Thanks a lot to AmbrosM (https://www.kaggle.com/ambrosm)

def cross_val(model, X, y):
    tic = time.time()
       
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_seed)
    
    roc_auc_score_te = []
    roc_auc_score_tr = []

    for fold, (indX_tr, indX_va) in enumerate(kf.split(X, y)):
        X_tr = X.iloc[indX_tr]
        X_va = X.iloc[indX_va]
        y_tr = y.iloc[indX_tr]
        y_va = y.iloc[indX_va]
        
        X_tr, X_va = scale_features(X_tr, X_va)

        model.fit(X_tr, y_tr)
        y_va_pred = model.predict_proba(X_va)
        y_tr_pred = model.predict_proba(X_tr)
        
        roc_auc_score_te.append(roc_auc_score(y_va, y_va_pred[:, 1]))
        roc_auc_score_tr.append(roc_auc_score(y_tr, y_tr_pred[:, 1]))
        
    roc_auc_score_te = np.array(roc_auc_score_te).mean()
    roc_auc_score_tr = np.array(roc_auc_score_tr).mean()
    
    tac= time.time()
    print(f'training score: {roc_auc_score_tr}')
    print(f'execution time of {model}: {round((tac-tic),2)} seconds')
    return roc_auc_score_tr, roc_auc_score_te

The following parameters are the result of Optima test that I did on another [notebook](https://www.kaggle.com/code/valerybonneau/ps4e01-optuna). There also few configurations that I found on notebook from others, unfortunately, I did not write down every contribution. Shame on me.

In [ ]:
best_params_cbc0= {'learning_rate': 0.010119738958122336,
                          'iterations': 56,
                          'depth': 3,
                          'min_child_samples': 18,
                          'subsample': 0.8025865440158371}
cbc0 = CatBoostClassifier(**best_params_cbc0)


best_params_cbc1= {'learning_rate': 0.04328357318741085,
                   'iterations': 691,
                   'depth': 5,
                   'min_child_samples': 39,
                   'subsample': 0.6809809689623991,
                   'colsample_bylevel': 0.6243594916266789,
                   'scale_pos_weight': 7.671793504598362,
                   'reg_lambda': 8.526571843521985}
cbc1 = CatBoostClassifier(**best_params_cbc1)

cbc2 = CatBoostClassifier(iterations=7500,od_type="Iter",eval_metric="AUC", random_seed=random_seed,early_stopping_rounds=2000)

best_params_xgbc0= {'learning_rate': 0.010003408261642718,
                           'n_estimators': 50,
                           'max_depth': 3,
                           'min_child_weight': 1.3295484808253735,
                           'subsample': 0.5330192943323063,
                           'colsample_bytree': 0.9840324890323718}
xgbc0 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc0)

best_params_xgbc1 = {'alpha': 0.21924507998693532,
   'colsample_bytree': 0.6190053199553928,
   'gamma': 0.20385269148738444,
   'lambda': 0.0007476286271033139,
   'learning_rate': 0.03361534115324951,
   'max_depth': 6,
   'min_child_weight': 5,
   'n_estimators': 427,
   'subsample': 0.607646482122379}
xgbc1 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc1)

best_params_xgbc2 = {
   'colsample_bytree': 0.5559356744724407,
   'gamma': 0.20385269148738444,
   'lambda': 0.0007476286271033139,
   'learning_rate': 0.14557589845509303,
   'max_depth': 4,
   'min_child_weight': 8.863137333171029,
   'n_estimators': 108}
xgbc2 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc2)


best_params_xgbc3 = {
    'n_estimators': 474,
    'learning_rate': 0.121800,
    'max_depth': 3,
    'num_leaves': 14,
    'min_child_samples': 2,
    'subsample':0.882688,
    'colsample_bytree': 0.822104,
    'reg_alpha':0.870367,
    'reg_lambda': 0.855034}
xgbc3 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc3)

best_params_xgbc4= {
    'tree_method': 'hist',
    'eval_metric': 'auc',
    'colsample_bytree': 0.40,
    'learning_rate': 0.06,
    'max_depth': 9,
    'n_estimators': 2500,                         
    'reg_alpha': 0.12,
    'reg_lambda': 0.8,
    'min_child_weight': 15,
}
xgbc4 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc4)


best_params_xgbc5= {
    'tree_method': 'hist',
    'eval_metric': 'auc',
    'colsample_bytree': 0.5,
    'learning_rate': 0.055,
    'max_depth': 9,
    'n_estimators': 3000,
    'reg_alpha': 0.2,
    'reg_lambda': 0.6,
    'min_child_weight': 25,
}
xgbc5 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc5)


best_params_xgbc6= {
    'tree_method': 'hist',
    'eval_metric': 'auc',
    'colsample_bytree': 0.80,
    'learning_rate': 0.082,
    'max_depth': 7,
    'n_estimators': 2000,
    'reg_alpha': 0.005,
    'reg_lambda': 0.95,
    'min_child_weight': 26,
}
xgbc6 = xgb.XGBClassifier(objective='binary:logistic', random_state=random_seed, **best_params_xgbc6)

best_params_xgbc7 = {
    'n_estimators': 2000,
    'max_depth': 6,
    'min_child_weight': 2.7527,
    'learning_rate': 0.01527,
    'subsample': 0.7109,
    'gamma': 0.2940,
    'colsample_bytree': 0.50157,
    'colsample_bylevel': 0.6877,
    'colsample_bynode': 0.94499}
xgbc7 = XGBClassifier(random_state=random_seed, **best_params_xgbc7)

best_params_lgbmc0 = {'n_estimators': 149,
                     'learning_rate': 0.06172,
                     'max_depth': 8,
                     'num_leaves': 32,
                     'min_child_samples': 7,
                     'subsample': 0.93605,
                     'colsample_bytree': 0.85325,
                     'reg_alpha': 0.567685, 
                     'reg_lambda': 0.913222}
lgbmc0 = LGBMClassifier(**best_params_lgbmc0, n_jobs=-1)


best_params_lgbmc1 = {'n_estimators': 327,
                      'learning_rate': 0.096359,
                      'max_depth': 6,
                      'num_leaves': 19,
                      'min_child_samples': 4,
                      'subsample': 0.676477,
                      'colsample_bytree': 0.76783,
                      'reg_alpha': 0.869556,
                      'reg_lambda': 0.6166327}
lgbmc1 = LGBMClassifier(**best_params_lgbmc1, n_jobs=-1)

best_params_lgbmc2 = {'learning_rate': 0.043283,
                      'iterations': 691,
                      'depth': 5,
                      'min_child_samples': 39,
                      'subsample': 0.680981,
                      'colsample_bytree': 0.6243591,
                      'scale_pos_weight': 7.671794,
                      'reg_lambda': 8.526572}

lgbmc2 = LGBMClassifier(**best_params_lgbmc2, n_jobs=-1)


best_params_lgbmc3 = {'n_estimators': 474,
                      'learning_rate': 0.12180036837974936,
                      'max_depth': 3,
                      'num_leaves': 14,
                      'min_child_samples': 2,
                      'subsample': 0.8826885927351279,
                      'colsample_bytree': 0.8221044666084394,
                      'reg_alpha': 0.8703670339049474,
                      'reg_lambda': 0.8550341657145066}

lgbmc3 = LGBMClassifier(**best_params_lgbmc3, n_jobs=-1)


best_params_lgbmc4 = {'n_estimators': 1464,
                      'max_depth': 41,
                      'learning_rate': 0.03433,
                      'min_child_weight': 2.96035,
                      'min_child_samples': 30,
                      'subsample': 0.878299,
                      'subsample_freq': 3,
                      'colsample_bytree': 0.501275,
                      'num_leaves': 25}
lgbmc4 = LGBMClassifier(random_state=random_seed,verbose=-1, **best_params_lgbmc4)

In [ ]:
model_names = {
    'cbc0': cbc0,
    'cbc1': cbc1,
    'cbc2': cbc2,
    'xgbc0': xgbc0,    
    'xgbc1': xgbc1,
    'xgbc2': xgbc2,
    'xgbc3': xgbc3,
    'xgbc4': xgbc4,    
    'xgbc5': xgbc5,
    'xgbc6': xgbc6,
    'xgbc7': xgbc7,
    'lgbmc0': lgbmc0,
    'lgbmc1': lgbmc1,
    'lgbmc2': lgbmc2,
    'lgbmc3': lgbmc3,
    'lgbmc4':lgbmc4
}

scores_model_bp = []
for model_name, model_instance in model_names.items():
    score_train, score_test = cross_val(model_instance, X_train_c, y_train)
    scores_model_bp.append((model_name, score_train, score_test))
    print(model_name, score_train, score_test)
scores_model_bp

<a id='six'><a/>
# 6. Neural Network Focus
Under construction

In [ ]:
X_train[['Gender', 'Country_Germany', 'Country_Spain']] = X_train[['Gender', 'Country_Germany', 'Country_Spain']].astype('int32')
X_test[['Gender', 'Country_Germany', 'Country_Spain']] = X_test[['Gender', 'Country_Germany', 'Country_Spain']].astype('int32')

In [ ]:
from keras.initializers import Constant

neurons = 64

lrelu = lambda x: keras.activations.relu(x, alpha=0.1)
leaky_relu = keras.layers.LeakyReLU(alpha=0.3)

model = keras.Sequential(
    [
        keras.layers.BatchNormalization(input_shape=[X_train.shape[1]], ),
        keras.layers.Dense(neurons, kernel_initializer='glorot_uniform', activation=None),
#         keras.layers.BatchNormalization(),
        keras.layers.Activation('swish'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(neurons, kernel_initializer='glorot_uniform', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('swish'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(neurons, kernel_initializer='glorot_uniform', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('swish'),
        keras.layers.Dropout(rate=0.1),
        keras.layers.Dense(1, activation='sigmoid')
    ])
model.summary()

In [ ]:
class_weight = (train['Exited'].value_counts(normalize=True)*100).round(2).tolist()
class_weight

In [ ]:
# adapted from https://www.kaggle.com/code/hridaym25/tensorflow-nn-with-lgbm-weights

class_weight_dic = {0: class_weight[0]/100, 1: class_weight[1]/100}

binary_accuracy_metric = tf.keras.metrics.BinaryAccuracy()
adam_optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False)
# clipvalue=1.0

def loss_fn(y_true, y_pred):
    return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)

def metric_fn(y_true, y_pred):
    return binary_accuracy_metric(y_true, y_pred)

model.compile(
    optimizer=adam_optimizer,
    loss=loss_fn,
    metrics=[metric_fn],
)

callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        patience=100, 
        restore_best_weights=True),
    tf.keras.callbacks.TensorBoard(log_dir="logs/{}", histogram_freq=1, profile_batch = 100000000)
]

history = model.fit(
    X_train,
    y_train,
    class_weight=class_weight_dic,
    callbacks=callbacks_list,
    validation_split=0.2,
    batch_size=32,
    epochs=35,
    verbose=2
)

In [ ]:
plt.plot(history.history['metric_fn'])
plt.plot(history.history['val_metric_fn'], label='val')
plt.title('AUC score')
plt.xlabel('epoch')
plt.ylabel('Auc')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='val')
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()
plt.show()

<a id='seven'></a>
# 7. Ensemble Model
I use hyper parameters that I've identified in that [notebook](https://www.kaggle.com/code/valerybonneau/ps4e01-optuna) using Optuna. Regarding the model that I try to optimize, I picked the one that got the best score with Autogluon and Bluecast. You can find the notebook [here](https://www.kaggle.com/code/valerybonneau/ps4e01-automl-test-with-bluecast-and-autogluon).

In [ ]:
sorted_score = sorted(scores_model_bp, key=lambda x: x[2], reverse=True)
best_score = sorted_score[:6]
best_estimators = [(model, model_names[model]) for model, _, _ in best_score]
best_score

In [ ]:
X_train = X_train_f[features_best_scores]
X_test = X_test_f[features_best_scores]

In [ ]:
X_train_fi, X_test_fi = scale_features(X_train_c, X_test_c)

In [ ]:
from sklearn.ensemble import VotingClassifier
estimators =( 
    best_estimators
    )

vr = VotingClassifier(estimators=estimators,
                     voting='soft',
                     verbose=False,
                      weights=[0.22, 0.20, 0.19, .16, 0.13, 0.10],
                     n_jobs=-1)
vr.fit(X_train_fi, y_train)
predictions = vr.predict_proba(X_test_fi)


<a id='eight'></a>
# 8. Submission and Conclusion
I ended up in the Top 15% as I wanted, learned quite a lot and applied different methods about feature importance.

In [ ]:
submission = pd.read_csv("/kaggle/input/playground-series-s4e3/sample_submission.csv")
submission.to_csv("submission.csv", index=False)